<a href="https://colab.research.google.com/github/lautitorraca/7506R-2C2023-GRUPO24/blob/main/TP2/7506R_TP2_GRUPO24_RED_NEURONAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

𝔾𝕣𝕦𝕡𝕠 𝟚𝟜 (Merequetengue👍)

➢ Torraca Lautaro - 108813

➢ Negrotti Gianluca - 108184

➢ Marco Tosi - 107237

### > Imports

In [32]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import BinaryAccuracy
import tensorflow as tf
from tensorflow.keras import backend as K

# Optimización de hiperparámetros
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, RepeatedKFold

# Manejo de modelos
from pickle import dump
from pickle import load

### > Lectura de los datasets

In [33]:
url_train = '/content/train_limpio.csv'
url_test = '/content/test_limpio.csv'

df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)

train = df_train.copy()
test = df_test.copy()

### Red neuronal

- Division y preprocesamiento de los datos



In [34]:
target = 'sentimiento'
features = 'review_es_clean'

In [35]:
X_train, X_test, y_train, y_test = train_test_split(train[features],
                                                    train[target],
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=train[target],
                                                    shuffle=True)



- Tokenizacion



In [36]:
# Convertir las críticas de texto a secuencias numéricas
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index

# Ajustar el tamaño del vocabulario en la capa Embedding
vocab_size = min(len(word_index) + 1, 20000)

# Limitar la longitud de las secuencias a un tamaño fijo
max_sequence_length = 200
X_train_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Convertir las etiquetas a valores numéricos
labels = np.array(y_train.replace({"negativo": 0, "positivo": 1}))


- Creacion del modelo de RNN

In [37]:
# Crear el modelo de RNN con capas GRU
model_opt = Sequential()
model_opt.add(Embedding(vocab_size, 100, input_length=max_sequence_length))
model_opt.add(GRU(64, dropout=0.2))  # Considera quitar recurrent_dropout si afecta el rendimiento en GPU
model_opt.add(Dense(1, activation='sigmoid'))

# Configurar el optimizador
optimizer = optimizers.Adam()

# Compilar el modelo
model_opt.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[BinaryAccuracy(name='accuracy')])

# Cantidad de épocas de entrenamiento
cant_epochs = 8





- Creacion de funcion para optimizar f1

In [38]:
def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


- Entrenamiento del modelo

In [ ]:
# Compilar el modelo
model_opt.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_score])

# Cantidad de épocas de entrenamiento
cant_epochs = 8

# Configurar el callback de Early Stopping para monitorear el f1_score
early_stopping = EarlyStopping(monitor='val_f1_score',
                               patience=4,
                               mode='max',
                               restore_best_weights=True,
                               min_delta=0.0001)

# Entrenar el modelo
history_model_rn = model_opt.fit(X_train_padded,
                                 labels,
                                 epochs=cant_epochs,
                                 batch_size=32,
                                 validation_split=0.2,
                                 verbose=1,
                                 callbacks=[early_stopping])


Epoch 1/8
875/875 [==============================] - 137s 154ms/step - loss: 0.4299 - f1_score: 0.7950 - val_loss: 0.3410 - val_f1_score: 0.8581
Epoch 2/8
644/875 [=====================>........] - ETA: 32s - loss: 0.2303 - f1_score: 0.9103

- Prediccion y muestra de la matriz

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


# Convertir las críticas de texto de prueba a secuencias numéricas
sequences_test = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(sequences_test, maxlen=max_sequence_length)

# Realizar predicciones
predictions = model_opt.predict(X_test_padded)

# Convertir las predicciones a etiquetas binarias
predicted_labels = [1 if p > 0.5 else 0 for p in np.ravel(predictions)]

# Convertir y_test a valores numéricos si es necesario
y_test_numeric = np.array(y_test.replace({"negativo": 0, "positivo": 1}))

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_numeric, predicted_labels)
print("Matriz de Confusión:")
print(conf_matrix)


- Guardado del modelo en H5 (similar a pickle)

In [ ]:
model_opt.save('red_neuronal.h5')  # Guarda el modelo en formato H5
from google.colab import files
files.download('red_neuronal.h5')


- Prediccion con el dataset Test

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convertir las revisiones de texto a secuencias numéricas
test_sequences = tokenizer.texts_to_sequences(test['review_es_clean'])


max_sequence_length = 200
X_test_tokenizer = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Realizar predicciones
predictions = model_opt.predict(X_test_tokenizer)
predicted_sentiment = ['positivo' if pred > 0.5 else 'negativo' for pred in predictions]

# Agregar las predicciones al DataFrame de prueba
test['predicted_sentiment'] = predicted_sentiment

# Seleccionar solo las columnas ID y predicted_sentiment
output = test[['ID', 'predicted_sentiment']]

# Guardar las predicciones en un nuevo archivo CSV
output.to_csv('test_predictions.csv', index=False)


- Descarga de la prediccion

In [ ]:
# Descargar el archivo (útil en Jupyter Notebook o Google Colab)
try:
    from google.colab import files
    files.download('test_predictions.csv')
except ImportError:
    print("La función de descarga solo funciona en entornos como Google Colab.")
